# Import

In [1]:
import numpy as np
import pandas as pd
from Bio import SeqIO

import torchvision.transforms as transforms

from torch.utils.data import DataLoader,Dataset
from torchvision import datasets
from torch.autograd import Variable 

import torch.nn as nn
import torch.nn.functional as F
import torch
from torch.autograd import Variable
import torch.autograd as autograd

from pathlib import Path

Сделать ваасермановский лосс
Разобраться с dimm - что ето?

In [2]:
import wandb

#wandb.login()
wandb.init(
    # set the wandb project where this run will be logged
    project="GAN-Z",
    
    # track hyperparameters and run metadata
    config={
    "learning_rate": 0.02,
    "architecture": "GAN",
    "dataset": "chr1",
    "epochs": 300,
    }

)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: xbostick. Use `wandb login --relogin` to force relogin


In [3]:
cuda = True if torch.cuda.is_available() else False # GPU Setting

# Setting parametrs

In [4]:
generic = False # Generate generic data on the fly (ignores data_loc and data_start args)
data_loc = Path("./data/") # Data location
epoch = 300
disc_iters = 5 # Number of iterations to train discriminator for at each epoch
latent_dim = 5 # Size of latent space
gen_dim = 100 # Generator dimension parameter
disc_dim = 100 # Discriminator dimension parameter
gen_layers = 5
disc_layer = 5
batch_size = 100
max_seq_len = 1000

 gen_data = lib.models.resnet_generator(latent_vars, args.gen_dim, args.max_seq_len, data_enc_dim, args.annotate)
  data_enc_dim = 4

# Preparing DATASET

In [5]:
dna_vocab = {"A":0,
             "C":1,
             "G":2,
             "T":3,
             "N":4} # catch-all auxiliary token

class GenomicData(Dataset):
    def __init__(self, path, seq_len):
        self.seq_len = seq_len
        with open(path) as fasta_file:  # Will close handle cleanly
            seq_record = {}
            seq_record["lengths"] = []
            seq_record["seq"] = []

            for seq_rec in SeqIO.parse(fasta_file, 'fasta'):  # (generator)
                seq_record["lengths"].append(len(seq_rec.seq))
                seq_record["seq"].append(seq_rec.seq)
        self.len = int(seq_record["lengths"][0])
        self.coded_seq = []
        for one in list(seq_record["seq"][0]):
            try:
                self.coded_seq.append(dna_vocab[one.upper()])
            except:
                print(one)

    def __len__(self):
        return int(self.len/self.seq_len)#int(np.round((self.len - self.seq_len)/100))
    
    def __getitem__(self, idx):
        return torch.Tensor(self.coded_seq[idx* self.seq_len:(idx+1)*self.seq_len])


GenData = GenomicData("./data/chr1.fa", max_seq_len)

# Structure of NNs

6 сверточных слоев с skip connection residual factor 0.3.
В сверточном слое 100 фильтров ядро 5 stribe 1
adam optimazer и wasserstein loss(WGAN) and lr 1e-5

In [6]:
import torch.nn as nn
import torch.functional as F

In [7]:
class Generator_new(nn.Module):
    def __init__(self,  latent_vars, gen_dim, max_seq_len, annotated=False, res_layers=5):
        super(Generator_new, self).__init__()
        self.input_size = latent_vars.shape
        self.output_size = gen_dim * max_seq_len
        self.seq_len = max_seq_len
        self.gen_dim = gen_dim
        self.res_layer = res_layers
        #self.conv1 = nn.Conv1d(3,6,5,1)
        self.conv1 = nn.Sequential(
                        nn.Conv1d(1,gen_dim,5,1, padding = 'same'),
                        nn.ReLU())
        self.con1 = nn.Conv1d(gen_dim ,gen_dim,5,1, padding = 'same')
        self.con2 = nn.Conv1d(gen_dim ,gen_dim,5,1, padding = 'same')
        self.con3 = nn.Conv1d(gen_dim ,gen_dim,5,1, padding = 'same')
        self.con4 = nn.Conv1d(gen_dim ,gen_dim,5,1, padding = 'same')
        self.con5 = nn.Conv1d(gen_dim ,gen_dim,5,1, padding = 'same')
        self.conv2 = nn.Sequential(
                        nn.Conv1d(gen_dim,1,5,1, padding = 'same'),
                        nn.ReLU(),
                        nn.Softmax())
        # self.con_array = [
        #     self.con1,
        #     self.con2,
        #     self.con3,
        #     self.con4,
        #     self.con5,
        # ]
        self.relu = nn.ReLU()
        self.sigmoid = nn.Softmax()#nn.Sigmoid()
        self.linear = nn.Linear(100,1)
        

    def forward(self,x):
        _input = self.conv1(x)#torch.reshape(x,(-1,self.gen_dim,self.seq_len))     
        
        out = self.con1(_input)
        out = self.relu(out)
        out = _input + 0.3 * out
        _input = out

        out = self.con1(_input)
        out = self.relu(out)
        out = _input + 0.3 * out
        _input = out

        out = self.con2(_input)
        out = self.relu(out)
        out = _input + 0.3 * out
        _input = out

        out = self.con3(_input)
        out = self.relu(out)
        out = _input + 0.3 * out
        _input = out

        out = self.con4(_input)
        out = self.relu(out)
        out = _input + 0.3 * out
        _input = out

        out = self.con5(_input)
        out = self.relu(out)
        out = self.sigmoid(_input + 0.3 * out)  
        # for i in range(self.res_layer):
        #     output = self.res_block(_input.float(),i, 0.3)
        #     _input = output
        #output = self.conv2(out)
        output = torch.transpose(out, 0, 1)
        output = self.linear(output) 
        return torch.transpose(output, 0, 1)
    
    def res_block(self,_input, index, downsample = 0):
        res = _input
        out = self.con_array[index](res)
        out = self.relu(out)
        if downsample:
            res = res * downsample
            out = out + res
        return out

In [8]:
class Critical_new(nn.Module):
    def __init__(self,  latent_vars, gen_dim, max_seq_len, annotated=False, res_layers=5):
        super(Critical_new, self).__init__()
        self.input_size = latent_vars.shape
        self.output_size = gen_dim * max_seq_len
        self.seq_len = max_seq_len
        self.gen_dim = gen_dim
        self.res_layer = res_layers
        #self.conv1 = nn.Conv1d(3,6,5,1)
        self.conv1 = nn.Conv1d(gen_dim,1,5,1, padding = 'same')
        self.con1 = nn.Conv1d(gen_dim ,gen_dim,5,1, padding = 'same')
        self.con2 = nn.Conv1d(gen_dim ,gen_dim,5,1, padding = 'same')
        self.con3 = nn.Conv1d(gen_dim ,gen_dim,5,1, padding = 'same')
        self.con4 = nn.Conv1d(gen_dim ,gen_dim,5,1, padding = 'same')
        self.con5 = nn.Conv1d(gen_dim ,gen_dim,5,1, padding = 'same')
        self.conv2 = nn.Sequential(
                        nn.Conv1d(1, gen_dim,5,1, padding = 'same'),
                        nn.ReLU())
        # self.con_array = [
        #     self.con1,
        #     self.con2,
        #     self.con3,
        #     self.con4,
        #     self.con5,
        # ]
        self.relu = nn.ReLU()
        # self.sigmoid = nn.Sigmoid()


    def forward(self,x):
        
        _input = x
        out = self.conv2(_input)
        
        out = self.con1(out)
        out = self.relu(out)
        out = _input + 0.3 * out
        _input = out

        out = self.con1(_input)
        out = self.relu(out)
        out = _input + 0.3 * out
        _input = out

        out = self.con2(_input)
        out = self.relu(out)
        out = _input + 0.3 * out
        _input = out

        out = self.con3(_input)
        out = self.relu(out)
        out = _input + 0.3 * out
        _input = out

        out = self.con4(_input)
        out = self.relu(out)
        out = _input + 0.3 * out
        _input = out

        out = self.con5(_input)
        out = self.relu(out)
        out = _input + 0.3 * out
        # for i in range(self.res_layer):
        #     output = self.res_block(_input.float(),i, 0.3)
        #     _input = output
        # #output = torch.reshape(output,(-1,1))

        out = self.conv1(out)
        return out #self.sigmoid(output)


In [9]:
num_classes = 10
num_epochs = 20
batch_size = 16
learning_rate = 0.01

In [10]:
latent_vars = torch.randn([10,10])

In [11]:
generator= Generator_new(latent_vars,gen_dim,max_seq_len)
discriminator = Critical_new(latent_vars,gen_dim,max_seq_len)

In [12]:
# generator.cuda()
# out = generator(z)
# discriminator.cuda()
# discriminator(out).size()

In [13]:
if cuda:
    generator.cuda()
    discriminator.cuda()
    #adversarial_loss.cuda()

_____

In [14]:
# suggested default - beta parameters (decay of first order momentum of gradients)
b1 = 0.5
b2 = 0.999

# suggested default - learning rate
lr = 1e-4

In [15]:
optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr, betas = (b1,b2) )
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr, betas = (b1,b2))

____

In [16]:
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

In [17]:
# latent_dim = 1
wandb.watch(generator, log_freq=100)
wandb.watch(discriminator, log_freq=100)

[]

In [18]:
def reset_grad(model):
    for param in model.parameters():
        param.grad = None


In [19]:
def compute_gradient_penalty(D, real_samples, fake_samples):
    """Calculates the gradient penalty loss for WGAN GP"""
    # Random weight term for interpolation between real and fake samples
    alpha = Tensor(np.random.random((real_samples.size(0), 1)))
        # Get random interpolation between real and fake samples
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_data))
    d_interpolates = discriminator(interpolates)
    fake = Variable(Tensor(1,1000).fill_(1.0), requires_grad=False)
        # Get gradient w.r.t. interpolates
    gradients = autograd.grad(
            outputs=d_interpolates,
            inputs=interpolates,
            grad_outputs=fake,
            create_graph=True,
            retain_graph=True,
            only_inputs=True,
        )[0]
    gradients = gradients.view(gradients.size(0), -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty

In [20]:
n_epochs = 200 # suggested default = 200
for epoch in range(n_epochs):
    dataloader = None
    dataloader = DataLoader(GenData,6,drop_last=True, shuffle = True)
    for i in tqdm(range(int(np.floor(len(GenData)/300)))):
        train_data = next(iter(dataloader)).type(Tensor)
        train_data.cuda()
        #   Critical forward-loss-backward-update
        for j in range(5):

            # Sample data
            noise = Variable(torch.randn((1,1000))).cuda()# Random sampling Tensor(batch_size, latent_dim) of Gaussian distribution
        
            real_data = torch.reshape(train_data[j],(1,1000))
            real_data = Variable(real_data)

            #generic
            fake_data = generator(noise)
            critics_real = discriminator(real_data)
            critics_fake = discriminator(fake_data)

            gradient_penalty = compute_gradient_penalty(discriminator, real_data, fake_data)
            critics_loss = -torch.mean(critics_real) + torch.mean(critics_fake) + 10 * gradient_penalty
            
            critics_loss.backward()
            optimizer_D.step()

            # Weight clipping
            for p in discriminator.parameters():
                p.data.clamp_(-0.01, 0.01)

            # Housekeeping - reset gradient
            reset_grad(generator)
            reset_grad(discriminator)

        #   Generator forward-loss-backward-update
        
        noise = Variable(torch.randn((1,1000))).cuda()# Random sampling Tensor(batch_size, latent_dim) of Gaussian distribution
        real_data = torch.reshape(train_data[j+1],(1,1000))
        real_data = Variable(real_data)
        #real_data = imgs["generator"].type(Tensor)

        fake_data = generator(noise)
        critics_fake = discriminator(fake_data)

        generator_loss = -torch.mean(critics_fake)
        
        generator_loss.backward()
        optimizer_G.step()

        # Housekeeping - reset gradient
        reset_grad(generator)
        reset_grad(discriminator)
    
    table = wandb.Histogram(np_histogram = np.histogram(generator(noise).detach().cpu().numpy()))
    wandb.log({"Epoch": epoch+1, "Critics_loss": critics_loss.item(), "Generator_loss": generator_loss.item(),"Results": table})

    print('D_loss: {}; G_loss: {}'
              .format(critics_loss.item(),  generator_loss.item()))


  0%|          | 0/829 [00:00<?, ?it/s]

C:\Temp\ipykernel_24080\1990929764.py:64: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  out = self.sigmoid(_input + 0.3 * out)


D_loss: 0.12158671021461487; G_loss: 0.017218194901943207


  0%|          | 0/829 [00:00<?, ?it/s]

D_loss: 0.05463799834251404; G_loss: 0.013123518787324429


...

  0%|          | 0/829 [00:00<?, ?it/s]

D_loss: 0.16662704944610596; G_loss: -0.007629492320120335


  0%|          | 0/829 [00:00<?, ?it/s]

D_loss: -0.02687123790383339; G_loss: -0.0033312449231743813


  0%|          | 0/829 [00:00<?, ?it/s]

In [ ]:
wandb.finish()

Critics_loss,▁▁▄▆▁▂██▄
Epoch,▁▂▃▄▅▅▆▇█
Generator_loss,███▇▅▅▅▃▁
Critics_loss,0.18323
Epoch,9
Generator_loss,-0.0262


In [ ]:
real_data.shape

torch.Size([1, 1000])

In [ ]:
noise = Variable(torch.randn((1,1000))).cuda()
generator(noise)

C:\Temp\ipykernel_19292\1990929764.py:64: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  out = self.sigmoid(_input + 0.3 * out)


tensor([[ -0.3573,  -0.8689,   0.2615,  -0.2626,  -0.2750,  -0.2750,  -0.2745,
          -0.2715,  -0.2750,  -0.2750,  -0.2743,  -0.2750,  -0.2750,  -0.2750,
          -0.2750,  -0.2750,  -0.2750,  -0.2750,  -0.2750,  -0.2750,  -0.2747,
          -0.2750,  -0.2750,  -0.2749,  -0.2749,  -0.2750,  -0.2750,  -0.2750,
          -0.2750,  -0.2750,  -0.2750,  -0.2750,  -0.2750,  -0.2750,  -0.2750,
          -0.2749,  -0.2750,  -0.2750,  -0.2750,  -0.2750,  -0.2750,  -0.2750,
          -0.2749,  -0.2750,  -0.2750,  -0.2750,  -0.2750,  -0.2750,  -0.2750,
          -0.2750,  -0.2750,  -0.2750,  -0.2750,  -0.2750,  -0.2748,  -0.2750,
          -0.2750,  -0.2750,  -0.2750,  -0.2749,  -0.2750,  -0.2750,  -0.2749,
          -0.2748,  -0.2750,  -0.2750,  -0.2633,  -0.2750,  -0.2750,  -0.2749,
          -0.2725,  -0.2750,  -0.2750,  -0.2748,  -0.2750,  -0.2750,  -0.2750,
          -0.2750,  -0.2750,  -0.2750,  -0.2750,  -0.2750,  -0.2750,  -0.2737,
          -0.2750,  -0.2750,  -0.2750,  -0.2748,  -0

In [ ]:
from torchvision import datasets
from torchvision.transforms import ToTensor

test_data = datasets.MNIST(root = 'data', train = False,
                           download = True, transform = ToTensor())

100%|██████████| 9912422/9912422 [00:00<00:00, 11439488.48it/s]


Extracting data\MNIST\raw\train-images-idx3-ubyte.gz to data\MNIST\raw



100%|██████████| 28881/28881 [00:00<00:00, 28732375.20it/s]


Extracting data\MNIST\raw\train-labels-idx1-ubyte.gz to data\MNIST\raw



100%|██████████| 1648877/1648877 [00:00<00:00, 12366876.77it/s]


Extracting data\MNIST\raw\t10k-images-idx3-ubyte.gz to data\MNIST\raw



100%|██████████| 4542/4542 [00:00<?, ?it/s]

Extracting data\MNIST\raw\t10k-labels-idx1-ubyte.gz to data\MNIST\raw



In [ ]:
a,b = test_data[0]

In [ ]:
load = DataLoader(test_data,batch_size = 10)

In [ ]:
next(iter(load))

[tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]]],
 
 
         [[[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]]],
 
 
         [[[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]]],
 
 
         ...,
 
 
         [[[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ..

In [ ]:
import torch
import torch.nn
import torch.nn.functional as nn
import torch.autograd as autograd
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os
from torch.autograd import Variable



mnist = test_data
mb_size = 28
z_dim = 10
X_dim = 28
y_dim = 1
h_dim = 128
cnt = 0
lr = 1e-4


G = torch.nn.Sequential(
    torch.nn.Linear(z_dim, h_dim),
    torch.nn.ReLU(),
    torch.nn.Linear(h_dim, X_dim),
    torch.nn.Sigmoid()
)
G.cuda()

D = torch.nn.Sequential(
    torch.nn.Linear(X_dim, h_dim),
    torch.nn.ReLU(),
    torch.nn.Linear(h_dim, 1),
)
D.cuda()

def reset_grad():
    G.zero_grad()
    D.zero_grad()


G_solver = optim.RMSprop(G.parameters(), lr=lr)
D_solver = optim.RMSprop(D.parameters(), lr=lr)


for it in range(1000000):
    for _ in range(5):
        # Sample data
        z = Variable(torch.randn(mb_size, z_dim))
        X,_ = (next(iter(load)))
        X = Variable(X).cuda()

        # Dicriminator forward-loss-backward-update
        G_sample = G(z.cuda())
        D_real = D(X)
        D_fake = D(G_sample)

        D_loss = -(torch.mean(D_real) - torch.mean(D_fake))

        D_loss.backward()
        D_solver.step()

        # Weight clipping
        for p in D.parameters():
            p.data.clamp_(-0.01, 0.01)

        # Housekeeping - reset gradient
        reset_grad()

    # Generator forward-loss-backward-update
    X, _ = next(iter(load))
    X = Variable(X).cuda()
    z = Variable(torch.randn(mb_size, z_dim)).cuda()

    G_sample = G(z)
    D_fake = D(G_sample)

    G_loss = -torch.mean(D_fake)

    G_loss.backward()
    G_solver.step()

    # Housekeeping - reset gradient
    reset_grad()

    #Print and plot every now and then
    if it % 1000 == 0:
        print('Iter-{}; D_loss: {}; G_loss: {}'
              .format(it, D_loss.cpu().data.numpy(), G_loss.cpu().data.numpy()))

        samples = G(z).cpu().data.numpy()#[:16]

        fig = plt.figure(figsize=(4, 4))
        gs = gridspec.GridSpec(4, 4)
        gs.update(wspace=0.05, hspace=0.05)

        
        plt.imshow(samples.reshape(28, 28), cmap='Greys_r')

        if not os.path.exists('out/'):
            os.makedirs('out/')

        plt.savefig('out/{}.png'.format(str(cnt).zfill(3)), bbox_inches='tight')
        cnt += 1
        plt.close(fig)

Iter-0; D_loss: -0.006369204260408878; G_loss: 0.004545770585536957
Iter-1000; D_loss: -0.004931438714265823; G_loss: -0.0005370582803152502
Iter-2000; D_loss: -0.003440557047724724; G_loss: -0.0007516610203310847
Iter-3000; D_loss: -0.0023505506105720997; G_loss: -0.0030477861873805523
Iter-4000; D_loss: -0.0015784932766109705; G_loss: -0.003214463358744979
Iter-5000; D_loss: -0.001160357380285859; G_loss: -0.0034737656824290752
Iter-6000; D_loss: -0.001041144598275423; G_loss: -0.002607758389785886
Iter-7000; D_loss: -0.0008465570863336325; G_loss: -0.0024224836379289627
Iter-8000; D_loss: -0.0010717147961258888; G_loss: -0.0033117288257926702
Iter-9000; D_loss: -0.0007010279223322868; G_loss: -0.0023659595753997564
Iter-10000; D_loss: -0.0005162782035768032; G_loss: -0.001913088490255177
Iter-11000; D_loss: -0.0006766270380467176; G_loss: -0.0031192232854664326
Iter-12000; D_loss: -0.0007853535935282707; G_loss: -0.002375855576246977
Iter-13000; D_loss: -0.0009073896799236536; G_los

KeyboardInterrupt: 

In [ ]:
layer = nn.Linear(100,1)

In [ ]:
noise = torch.randn(1000, 100)
layer(noise).shape

torch.Size([1000, 1])

In [ ]:
noise.transpose()

TypeError: transpose() received an invalid combination of arguments - got (), but expected one of:
 * (int dim0, int dim1)
 * (name dim0, name dim1)


In [ ]:
torch.transpose(noise, 0, 1).shape

torch.Size([100, 1000])

In [ ]:
class SkipConnection(nn.Module):
    def __init__(self, alpha, original):
        super().__init__()
        self.alpha =  alpha
    
    def forward(self, x):
        x = x
        return x

In [ ]:
class Generator_new(nn.Module):
    def __init__(self,  latent_vars, gen_dim, max_seq_len, annotated=False, res_layers=5):
        super(Generator_new, self).__init__()
        self.input_size = latent_vars.shape
        self.output_size = gen_dim * max_seq_len
        self.seq_len = max_seq_len
        self.gen_dim = gen_dim
        self.res_layer = res_layers
        #self.conv1 = nn.Conv1d(3,6,5,1)
        self.conv1 = nn.Sequential(
                        nn.Conv1d(1,gen_dim,5,1, padding = 'same'),
                        nn.ReLU(),
                        nn.Sigmoid())
        self.con1 = nn.Conv1d(gen_dim ,gen_dim,5,1, padding = 'same')
        self.con2 = nn.Conv1d(gen_dim ,gen_dim,5,1, padding = 'same')
        self.con3 = nn.Conv1d(gen_dim ,gen_dim,5,1, padding = 'same')
        self.con4 = nn.Conv1d(gen_dim ,gen_dim,5,1, padding = 'same')
        self.con5 = nn.Conv1d(gen_dim ,gen_dim,5,1, padding = 'same')
        self.conv2 = nn.Sequential(
                        nn.Conv1d(gen_dim,1,5,1, padding = 'same'),
                        nn.ReLU(),
                        nn.Sigmoid())
        # self.con_array = [
        #     self.con1,
        #     self.con2,
        #     self.con3,
        #     self.con4,
        #     self.con5,
        # ]
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

        G = torch.nn.Sequential(
            nn.Conv1d(1,gen_dim,5,1, padding = 'same'),
            nn.ReLU(),
            nn.Conv1d(gen_dim ,gen_dim,5,1, padding = 'same'),
            
            torch.nn.Sigmoid()
        )

    def forward(self,x):
        _input = self.conv1(x)#torch.reshape(x,(-1,self.gen_dim,self.seq_len))     
        
        out = self.con1(_input)
        out = self.relu(out)
        out = _input + 0.3 * out
        _input = out

        out = self.con1(_input)
        out = self.relu(out)
        out = _input + 0.3 * out
        _input = out

        out = self.con2(_input)
        out = self.relu(out)
        out = _input + 0.3 * out
        _input = out

        out = self.con3(_input)
        out = self.relu(out)
        out = _input + 0.3 * out
        _input = out

        out = self.con4(_input)
        out = self.relu(out)
        out = _input + 0.3 * out
        _input = out

        out = self.con5(_input)
        out = self.relu(out)
        out = _input + 0.3 * out   
        # for i in range(self.res_layer):
        #     output = self.res_block(_input.float(),i, 0.3)
        #     _input = output
        output = self.conv2(_input)
        #output = self.relu(output)
        return self.sigmoid(output)
    
    def res_block(self,_input, index, downsample = 0):
        res = _input
        out = self.con_array[index](res)
        out = self.relu(out)
        if downsample:
            res = res * downsample
            out = out + res
        return out
class Critical_new(nn.Module):
    def __init__(self,  latent_vars, gen_dim, max_seq_len, annotated=False, res_layers=5):
        super(Critical_new, self).__init__()
        self.input_size = latent_vars.shape
        self.output_size = gen_dim * max_seq_len
        self.seq_len = max_seq_len
        self.gen_dim = gen_dim
        self.res_layer = res_layers
        #self.conv1 = nn.Conv1d(3,6,5,1)
        self.conv1 = nn.Sequential(
                        nn.Conv1d(gen_dim,1,5,1, padding = 'same'),
                        nn.ReLU())
        self.con1 = nn.Conv1d(gen_dim ,gen_dim,5,1, padding = 'same')
        self.con2 = nn.Conv1d(gen_dim ,gen_dim,5,1, padding = 'same')
        self.con3 = nn.Conv1d(gen_dim ,gen_dim,5,1, padding = 'same')
        self.con4 = nn.Conv1d(gen_dim ,gen_dim,5,1, padding = 'same')
        self.con5 = nn.Conv1d(gen_dim ,gen_dim,5,1, padding = 'same')
        self.conv2 = nn.Sequential(
                        nn.Conv1d(1, gen_dim,5,1, padding = 'same'),
                        nn.ReLU())
        # self.con_array = [
        #     self.con1,
        #     self.con2,
        #     self.con3,
        #     self.con4,
        #     self.con5,
        # ]
        self.relu = nn.ReLU()
        # self.sigmoid = nn.Sigmoid()


    def forward(self,x):
        
        _input = x
        out = self.conv2(_input)
        
        out = self.con1(out)
        out = self.relu(out)
        out = _input + 0.3 * out
        _input = out

        out = self.con1(_input)
        out = self.relu(out)
        out = _input + 0.3 * out
        _input = out

        out = self.con2(_input)
        out = self.relu(out)
        out = _input + 0.3 * out
        _input = out

        out = self.con3(_input)
        out = self.relu(out)
        out = _input + 0.3 * out
        _input = out

        out = self.con4(_input)
        out = self.relu(out)
        out = _input + 0.3 * out
        _input = out

        out = self.con5(_input)
        out = self.relu(out)
        out = _input + 0.3 * out
        # for i in range(self.res_layer):
        #     output = self.res_block(_input.float(),i, 0.3)
        #     _input = output
        # #output = torch.reshape(output,(-1,1))

        out = self.conv1(out)
        return out #self.sigmoid(output)
    
    def res_block(self,_input, index, downsample = 0):
        res = _input
        out = self.con_array[index](res)
        out = self.relu(out)
        if downsample:
            res = res * downsample
            out = out + res
        return out